In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import boto3
import pyarrow
import pymysql
import mysql.connector
from datetime import datetime, timezone, timedelta
from decouple import config

In [2]:
s3 = boto3.resource(
    service_name ='s3',
    region_name = config('AWS_REGION'),
    aws_access_key_id = config('AWS_KEY'),
    aws_secret_access_key = config('AWS_SECRET')
)

In [3]:
for bucket in s3.buckets.all():
    print(bucket.name)

    

mygamelogsbucket
thebucketofjacob2020


In [4]:
# READ a file directly from s3.  no download.
obj = s3.Bucket('mygamelogsbucket').Object('bby.csv').get()
s3_gamelogs = pd.read_csv(obj['Body'], index_col = 0) # removes the useless index column

In [5]:
### #SQL DATABASE STUFF
mydb = sqlalchemy.create_engine('mysql+mysqlconnector://' + config('USER') + ':' + config('PW') + '@' + config('IP') + ':' + config('PORT') + '/' + config('DB'),
                     echo = False)


In [6]:
bb1 = pd.read_sql('SELECT * FROM aws_gamelogs;', con = mydb)
bb2 = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = mydb)


In [7]:
cnx = mydb.raw_connection()

In [8]:
connection = mysql.connector.connect(host = config('IP'),
                                     port = config('PORT'),
                                      user = config('USER'),
                                      password = config('PW'),
                                      db = config('DB'))


# connection

all_tables = pd.read_sql_query('SHOW TABLES FROM aws_database;', connection)

In [9]:
df1 = pd.read_sql('SELECT * FROM aws_gamelogs;', con = connection)
df2 = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = connection)

In [10]:
gamelogs = pd.read_sql_query('SELECT * FROM aws_gamelogs;', connection)

gamelogs_new = gamelogs.groupby(['Team', 'Type', 'GameID']).mean('PTS')

gamelogs_type_aggs = gamelogs[(gamelogs['Type'] != 'Play-In Tournament')] \
    .groupby(['Team', 'Type', 'GameID']) \
    .sum()['PTS'] \
    .groupby(['Team', 'Type']) \
    .mean().round(1) \
    .reset_index() \
    .rename(columns = {'PTS' : 'team_pts'}) \
    .sort_values('team_pts', ascending = False) \
    .assign(language = 'Python7-4')

In [11]:
# gamelogs_type_aggs.to_sql(con = connection, name = "my_python_table", index = False, if_exists = "replace")
# doesnt work, cant use connection var

In [12]:
gamelogs_type_aggs.to_sql(con = mydb, name = "my_python_table", index = False, if_exists = "replace")


In [13]:
new_df = pd.read_sql_query('SELECT * FROM my_python_table;', con = mydb)

In [14]:
# i can read sql with both mysql connector and sqlalchemy engine
# but can only write to rds with sqlalchemy engine

In [15]:
# start_time = datetime.utcnow()
# my_time = datetime.now().time()
# current_datetime = datetime.today().strftime("%Y-%m-%d %H:%M:%S %p %Z")
# current_date = datetime.now().date()
# print(current_date)
# print(current_datetime)
# print('starting function at', start_time)
yesterday = datetime.now().date() - timedelta(1)

In [16]:
# top 15 scorers from most recent games
df = pd.read_sql_query('SELECT * FROM aws_gamelogs;', con = mydb)

df['Date'] = pd.to_datetime(df['Date']).dt.date

In [17]:
min(df['Date'])
max(df['Date'])

datetime.date(2021, 7, 6)

In [20]:
top10_yesterday = df[df['Date'] == max(df['Date'])] \
    .nlargest(10, 'PTS') \
    .sort_values('PTS', ascending = False)
# top10_yesterday = df2[['Player', 'Date', 'Outcome', 'Team', 'PTS']]

In [25]:
df.groupby('Team').sum('PTS')

KeyError: ('Team', 'PTS')